# Random Forest Classification

Random forest classification is a modified tree learning algorithm.

Hyperparameters:

- Number of trees used
- number of features
- number of samples --> default of which is the training set

In [53]:
import pandas as pd
import numpy as np
import os
from pprint import pprint

from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import cross_validate
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

def run_classifier(file_path, threshold=0.6):
    print("Loading data...")
    try:
        data = pd.read_csv(file_path)
        print("Data loaded successfully from {file_path}")
        print(data)
    except Exception as e:
        print("Error loading data: {e}")
        return

run_classifier("C:\\btc_usd.csv")

Loading data...
Data loaded successfully from {file_path}
                           Date          Open          High           Low  \
0     2014-09-17 00:00:00+00:00    465.864014    468.174011    452.421997   
1     2014-09-18 00:00:00+00:00    456.859985    456.859985    413.104004   
2     2014-09-19 00:00:00+00:00    424.102997    427.834991    384.532013   
3     2014-09-20 00:00:00+00:00    394.673004    423.295990    389.882996   
4     2014-09-21 00:00:00+00:00    408.084991    412.425995    393.181000   
...                         ...           ...           ...           ...   
3726  2024-11-29 00:00:00+00:00  95653.953125  98693.171875  95407.882812   
3727  2024-11-30 00:00:00+00:00  97468.812500  97499.343750  96144.218750   
3728  2024-12-01 00:00:00+00:00  96461.335938  97888.125000  95770.187500   
3729  2024-12-02 00:00:00+00:00  97276.007812  98152.601562  94482.867188   
3730  2024-12-03 00:00:00+00:00  95855.929688  96263.234375  93645.632812   

             Clos